In [8]:
import numpy as np
import matplotlib.pyplot as plt
from torch import Tensor
from torch.nn import Linear, CrossEntropyLoss, MSELoss
from torch.optim import LBFGS
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector
import torch
from torch import cat, no_grad, manual_seed
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.optim as optim
from torch.nn import (
    Module,
    Conv2d,
    Linear,
    Dropout2d,
    NLLLoss,
    CrossEntropyLoss,
    MaxPool2d,
    Flatten,
    Sequential,
    ReLU,
)
import torch.nn.functional as F
from qiskit.primitives import StatevectorEstimator as Estimator
import torch.nn as nn
from torchvision import models

In [1]:
batch_size = 2
n_samples = 100 # dataset size (train)

num_inputs = 2 # num of features (n)
num_params = 12 # num of parameters (m)
num_qubits = 2 # num of qubits

In [2]:
# load MNIST train data
X_train = datasets.MNIST(
    root="./data",
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor()])
)

# Filter out labels to keep only labels 0 and 1
idx = np.append(
    np.where(X_train.targets == 0)[0][:n_samples],
    np.where(X_train.targets == 1)[0][:n_samples]
)
X_train.data = X_train.data[idx]
X_train.targets = X_train.targets[idx]

train_loader = DataLoader(X_train, batch_size=batch_size, shuffle=True)

In [ ]:
def create_qcc(num_qubits, num_inputs, num_params):

  inputs = [Parameter(f"x{i}") for i in range(1, num_inputs+1)] 
  params = [Parameter(f"theta{i}") for i in range(1, num_params+1)]

  qcc = QuantumCircuit(num_qubits) 

  x1 = inputs[0]
  x2 = inputs[1]
  theta1 = params[0]
  theta2 = params[1]
  theta3 = params[2]
  theta4 = params[3]
  theta5 = params[4]
  theta6 = params[5]
  theta7 = params[6]
  theta8 = params[7]
  theta9 = params[8]
  theta10 = params[9]
  theta11 = params[10]
  theta12 = params[11]

  qcc.rx(x1, 0)
  qcc.rx(x2, 1)
  qcc.rzz(theta1, 0, 1)
  qcc.ry(theta2, 0)
  qcc.ry(theta3, 1)
  qcc.rx(x1, 0)
  qcc.rx(x2, 1)
  qcc.rzz(theta4, 0, 1)
  qcc.ry(theta5, 0)
  qcc.ry(theta6, 1)
  qcc.rx(x1, 0)
  qcc.rx(x2, 1)
  qcc.rzz(theta7, 0, 1)
  qcc.ry(theta8, 0)
  qcc.ry(theta9, 1)
  qcc.rx(x1, 0)
  qcc.rx(x2, 1)
  qcc.rzz(theta10, 0, 1)
  qcc.ry(theta11, 0)
  qcc.ry(theta12, 1)
  qcc.rx(x1, 0)
  qcc.rx(x2, 1)

  return qcc, inputs, params

In [9]:
estimator = Estimator()

def create_qnn():
    
    qcc, inputs, params = create_qcc(num_qubits, num_inputs, num_params)
    qnn = EstimatorQNN(
        circuit=qcc,
        input_params=inputs,
        weight_params=params,
        input_gradients=True,
        estimator=estimator,
    )
    return qnn

qnn = create_qnn()

In [ ]:
# Basic neural network implementation

class Net(Module):
    def __init__(self, qnn):
        super().__init__()
        self.conv1 = Conv2d(1, 2, kernel_size=5)
        self.conv2 = Conv2d(2, 16, kernel_size=5)
        self.dropout = Dropout2d()
        self.fc1 = Linear(256, 64)
        self.fc2 = Linear(64, 2)  # 2-dimensional input to QNN
        self.qnn = TorchConnector(qnn) 
        self.fc3 = Linear(1, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout(x)
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.qnn(x)  # apply QNN
        x = self.fc3(x)
        return x

In [12]:
# ResNet based implementation

class ResNetQNN(nn.Module):
    def __init__(self, qnn):
        super(ResNetQNN, self).__init__()

        self.resnet = models.resnet18(pretrained=True)
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)  # Modify input layer to accept grayscale images
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Identity()  # Remove the final FC layer
        self.fc1 = nn.Linear(num_features, 2)  # reduce dim to 2: input to QNN

        self.qnn = TorchConnector(qnn)  # quantum layer

        # Final classification layer to output 2 logits for binary classification
        self.fc2 = nn.Linear(1, 2)  # final classif layer 

    def forward(self, x):
        x = self.resnet(x)  
        x = F.relu(self.fc1(x))
        x = self.qnn(x) 
        x = self.fc2(x)

        return x  # Return logits for 2 classes (0 and 1)

In [ ]:
# choose one of the two models
model = ResNetQNN(qnn)
# model = Net(qnn)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.0005)
loss_func = CrossEntropyLoss()

# Start training
epochs = 20  
loss_list = [] 
model.train() 

for epoch in range(epochs):
    total_loss = []
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad(set_to_none=True)  # Initialize gradient
        output = model(data)  # Forward pass
        loss = loss_func(output, target)  # Calculate loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize weights
        total_loss.append(loss.item())  
    loss_list.append(sum(total_loss) / len(total_loss))
    print("Training [{:.0f}%]\tLoss: {:.4f}".format(100.0 * (epoch + 1) / epochs, loss_list[-1]))


In [ ]:
# Plot loss convergence
plt.plot(loss_list)
plt.title("Hybrid NN Training Convergence")
plt.xlabel("Training Iterations")
plt.ylabel("Neg. Log Likelihood Loss")
plt.show()